# Extract the zones and origin-destination matrix for visualization purpose

Region: The Netherlands

Purpose: To develope a data product that explores the origin-destination matrix using the travel survey.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import subprocess
import sys
import pandas as pd

def get_repo_root():
    """Get the root directory of the repo."""
    dir_in_repo = os.path.dirname(os.path.abspath('__file__')) # os.getcwd()
    return subprocess.check_output('git rev-parse --show-toplevel'.split(),
                                   cwd=dir_in_repo,
                                   universal_newlines=True).rstrip()
sys.path.append(get_repo_root())
ROOT_dir = get_repo_root()

import lib.netherlands as netherlands

## 1 Load zones

In [2]:
ground_truth = netherlands.GroundTruthLoader()

# load zones
ground_truth.load_zones()


## 2 Load odms by mode

In [3]:
odm_list = []
for m_id, m_name in zip([0, 1, 2, 4], ['Non-bike', 'E-bike', 'Bike', 'Total']):
    ground_truth.load_odm_visualization(mode=m_id)
    ser = ground_truth.odm
    ser.name = m_name
    odm_list.append(ser)

origin_zip  dest_zip
0           0           1.545977e+08
            1012        3.363486e+05
            1015        3.759916e+05
            1016        9.227314e+04
            1017        7.002134e+05
Name: weight_trip, dtype: float64
zone  zone
1011  1011    1.375448e+06
      1012    5.502692e+05
      1013    0.000000e+00
      1014    0.000000e+00
      1015    0.000000e+00
Name: weight_trip, dtype: float64
origin_zip  dest_zip
0           0           3.609388e+06
            1035        1.935871e+05
            1071        1.528667e+05
            1131        6.050981e+04
            1611        2.184031e+05
Name: weight_trip, dtype: float64
zone  zone
1011  1011    0.0
      1012    0.0
      1013    0.0
      1014    0.0
      1015    0.0
Name: weight_trip, dtype: float64
origin_zip  dest_zip
0           0           1.681924e+07
            1011        3.320952e+05
            1013        1.961443e+05
            1015        2.006701e+05
            1016        1.324253e+05

## 3 Combine all the odms

In [4]:
# Combine mode-specific odms
df = pd.concat(odm_list, axis=1)

# Remove zero cells
df = df.loc[(df!=0).any(axis=1)]

# Rename MultiIndex and reset
df.index.names = ['ozone', 'dzone']
df = df.reset_index()

# Save ODMs and zones
df.to_csv(ROOT_dir + '/dbs/netherlands/odms_vis/odms_by_mode.csv', index=False)
ground_truth.zones.to_file(ROOT_dir + '/dbs/netherlands/odms_vis/zones.shp')

ValueError: cannot insert zone, already exists